In [1]:
from yolo_detector import arg_parse, load_yolo_model, detect_image , draw_boxes, prepare_image
from util import write_results
from yolo_attacks import reverse_prepare_image,dag_attack
import torch

In [2]:
cfgfile = "cfg/yolov3.cfg"
reso = 416
weightsfile = "yolov3.weights"
images = "coco/test_images/000000000001.jpg"
batch_size = 1
confidence = 0.5
nms_thesh = 0.4
CUDA = torch.cuda.is_available()
det = "detections"

In [3]:
model, classes, num_classes, inp_dim = load_yolo_model(cfgfile,weightsfile,reso ,CUDA = CUDA)

Loading network.....
Network successfully loaded


In [8]:
output, imlist, loaded_ims = detect_image(model, classes, num_classes, inp_dim, images, batch_size, confidence, nms_thesh,det, CUDA = CUDA)
draw_boxes(loaded_ims,output,imlist,classes, det)

----------------------------------------------------------
Objects Detected:    car car truck parking meter
----------------------------------------------------------
Detection Images are saved in detections


In [5]:
target_objects = [2,7]  # car and truck
adversarial_class = [0,1] #person and bicycle
adv_images, _, im_dim_list = dag_attack(
    model, 
    images, 
    target_objects, 
    adversarial_class, 
    max_iter=10, 
    gamma=0.1, 
    CUDA=CUDA
)

Step 1/10: Loss = 433020.4375
Step 2/10: Loss = 784581.625
Step 3/10: Loss = 599826.875
Step 4/10: Loss = 225695.71875
Step 5/10: Loss = 18756.810546875
Step 6/10: Loss = 10088.7080078125
Step 7/10: Loss = 781.73974609375
Step 8/10: Loss = 190354.203125
Step 9/10: Loss = 10180.6337890625
Step 10/10: Loss = 9982.9658203125


In [9]:
reverse_prepare_image(adv_images, im_dim_list, "adversarial_images", inp_dim)

Restored images saved in adversarial_images


In [10]:
output, imlist, loaded_ims = detect_image(model, classes, num_classes, inp_dim, "adversarial_images", batch_size, confidence, nms_thesh,"adversarial_detections", CUDA = CUDA)
draw_boxes(loaded_ims,output,imlist,classes, "adversarial_detections")

----------------------------------------------------------
Objects Detected:    person person train truck
----------------------------------------------------------
Detection Images are saved in adversarial_detections
